
$\qquad$ $\qquad$$\qquad$  **TDA231/DIT370 Discrete Optimization: Home Assignment 4 -- SDP and Maxcut** <br />
$\qquad$ $\qquad$$\qquad$                   **Grader: Divya** <br />
$\qquad$ $\qquad$$\qquad$                     **Due Date: 11th March** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Qufei Wang, 900212-6952., qufei@student.chalmers.se <br /> Markus Ingvarsson, 940712-0154, markusi@student.chalmers.se


---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook, and equations wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook do not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Question 1.

Consider the triangle graph i.e, three vertices all connected pairwise (unweighted, so each edge has weight $1$).

* a. Write the $+1/-1$ labelling corresponding to a maximum cut and give the value of the cut.
* b. Write the SDP relaxation of the MAXCUT problem for this specific graph: write the program explicitly without using summation signs, with the variables corresponding to a $3 \times 3$ matrix, $X_{1,1,}, X_{1,2}, \cdots$.
* c. Solve the SDP by manual calculation (not using a SDP solver). (HINT: Use symmetry and then argue about when the matrix with $1$ as diagonal elements and $\alpha$ in other positions is psd.)
* d. Produce the vector labelling corresponding to the optimal solution of the SDP in (c). (HINT: for what angle is $\cos \theta = -1/2$?)
* e. What is the expected value of the cut produced by rounding using the vector labels and a random hyperlane? Give a short justification.


1.a We represent the triangle graph as $G=(V,E)$, where $V = {1, 2, 3}$ and $E = {(1, 2), (1, 3), (2, 3)}$. Then a labelling of maximum cut would be giving vertices $v_1, v_2$ value $1$ and $v_3$ value $-1$. Then the value of the maximum cut is $2$. In fact, for a triangle graph, there are only 3 ways to partition the vertices, one can verify that any partition yields in a result cut of value 2.

1.b The SDP relaxation of the MAXCUT problem for this specific graph is as follows:

$\begin{array}{cl}\text { Maximize } & \frac{1 - x_{12}}{2} + \frac{1 - x_{13}}{2} + \frac{1 - x_{23}}{2} \\ \text { subject to } & x_{i i}=1, \quad i=1,2, \ldots, n \\ & X \succeq 0\end{array}$

1.c Suppose we have the variable matrix $A$ of our SDP in this form:
\begin{pmatrix}
1 & \alpha & \alpha \\
\alpha & 1 & \alpha \\
\alpha & \alpha & 1 \\
\end{pmatrix}
We use the definition of PSD that all the eigenvalues of our variable matrix $A$ are nonnegetive. The eigenvalues $\lambda$ of matrix $A$ can be found by:
\begin{equation}
det(A - \lambda I) = 0
\end{equation}
which is equivalent to
$$ 
\begin{vmatrix}
1 - \lambda & \alpha & \alpha \\
\alpha & 1 - \lambda & \alpha \\
\alpha & \alpha & 1 - \lambda \\
\end{vmatrix} = 0
$$
Calculating this determinant, we have 
$$ (1 + 2\alpha - \lambda)(1 - \alpha - \lambda)^2 = 0$$
So we have the eigenvalues $\lambda$ of the form
$$ \lambda = 1 - \alpha, \lambda = 1 + 2\alpha$$
Because $\lambda$ is nonnegetive, we get the constraints on $\alpha$ : $-0.5 \leq \alpha \leq 1$.

In order to make our objective value $\frac{1 - x_{12}}{2} + \frac{1 - x_{13}}{2} + \frac{1 - x_{23}}{2}$ as large as possible, we choose $\alpha = -0.5$, which is also the value of $x_{12},x_{13},x_{23}$. Thus we have the optimal value $\frac{1.5}{2} * 3 = 2.25$

1.d A vector labelling of the optimal solution of the SDP in c could be:
$$v_1 = (0, 1, 0), v_2 = (-\frac{\sqrt{3}}{2}, -\frac{1}{2}, 0), v_3 = (\frac{\sqrt{3}}{2}, -\frac{1}{2}, 0)$$

1.e The expected value of cut by rounding is 2. The three vectors we choose in question $d$ are in the same subspace and have the same angle between them with value $2\pi/3$. For any randomly choosen vector $p$ in the 2-dimensional unit sphere, we consider the projection $r$ it has on the subspace determined by our three choosen vectors. It turns out that by using the rounding rule,
$$
\mathbf{u} \mapsto\left\{\begin{aligned} 1 & \text { if } \mathbf{p}^{T} \mathbf{u} \geq 0 \\-1 & \text { otherwise } \end{aligned}\right.
$$
we either map one or two of our labelling vectors to value $1$. We can not map all of them to $1$ or $-1$. Therefore we always end up with a result of cut 2.


# Question 2.


* a. Implement the SDP relaxation for MAXCUT in CVXPY (https://www.cvxpy.org), see (https://www.cvxpy.org/examples/basic/sdp.html). (Unfortunately  the API for SDP in CVXOPT is not very convenient, so we are forced to use CVXPY. The best way if you have installation issues would be to use Google Colab, there you can directly start using it.)
* b. Solve the SDP in the previous problem using (a).
* c. Solve the SDP and give the MAXCUT value for for the random graph $G(100, 0,1)$. Give the approximation ratio of your solution to the optimal max-cut value.
* d. Solve the SDP and give the MAXCUT value for the planted partition graph 'G' given below. Give the approximation ratio.

In [1]:
import cvxpy as cp
import numpy as np
import random

# 2.a
# used to express the constraints on variable matrix, which is the diagonal entries of the matrix being 1.
def singleNonZeroMatrix(i, n):
    a = np.zeros((n, n), float)
    a[i, i] = 1
    return a
# used to express the objective value of the sdp relaxation for the 'maxcut' problem.
# 'edges' is the list of edges of the graph expressed by tuples, for example [(0, 1), (1, 2)] means there's an edge
# between the first and the second vertex, and an edge between the second and third vertex.
def objectiveValueMatrix(n, edges):
    a = np.zeros((n, n), float)
    for i in range(len(edges)):
        row = edges[i][0]
        col = edges[i][1]
        a[row, col] = 0.5
    return a
    
# the implementation of SDP relaxation for MAXCUT in CVXPY
def sdp_maxcut(n, edges):
    X = cp.Variable((n,n), symmetric=True)
    constraints = [X >> 0]
    constraints += [cp.trace(singleNonZeroMatrix(i, n) @ X) == 1.0 for i in range(n)]
    C = objectiveValueMatrix(n, edges)
    prob = cp.Problem(cp.Minimize(cp.trace(C @ X) - len(edges)/2),
                  constraints)
    prob.solve()
    return prob, X
    

# 2.b
edges = [(0, 2), (0, 1), (1, 2)]
prob, X = sdp_maxcut(3, edges)
print("The optimal value for Question 1 using 2.a is", (-1) * prob.value)
print("A solution X is")
print(X.value)


# 2.c
def toEdges():
    edges = []
    f = open("graph.txt","r")
    text = f.read()
    lines = text.splitlines()
    for i in range(len(lines)):
        line = lines[i]
        a = np.fromstring(line, dtype=int, sep=' ')
        for j in range(i, len(a)):
            if a[j] == 1:
                edges.append((i,j))       
    return edges

def vectorNorm(size): 
    a = np.random.rand(1, size)
    b = np.linalg.norm(a)
    return a/b


def rounding(m, p):
    r = []
    col = m.shape[1]
    for i in range(col):
        t = np.matmul(p, m[:, i])
        if t >= 0:
            r.append(1)
        else:
            r.append(-1)
    return r

def decompose(X):
    size = X.shape[0]
    Xnew = X.value
    eigs = np.linalg.eigh(Xnew)[0]
    if np.min(eigs) < 0:
      Xnew = Xnew + (1.00001 * abs(min(eigs)) * np.identity(size))
    elif np.min(eigs) == 0:
      Xnew = Xnew + 0.0000001 * np.identity(size)
    return np.linalg.cholesky(Xnew).T
    
def calCut(edges, vertices):
    v = 0
    for i in range(len(edges)):
        edge = edges[i]
        v += (1 - vertices[edge[0]] * vertices[edge[1]]) / 2
    return v
 
def maxCut():
    edges = toEdges()    
    prob, X = sdp_maxcut(100, edges)
    print("The optimal value of the SDP program in 2.c is ", (-1) * prob.value)

    U = decompose(X)
    val = 0
    ratio = None
    ite = 10
    for i in range(ite):
        p = vectorNorm(100)
        vertices = rounding(U, p)
        val += calCut(edges, vertices)
    val = val / 10
    ratio = val / ((-1) * prob.value)
    return val, ratio

val, ratio = maxCut()
print("The optimal value of the MAXCUT in 2.c is: " + str(val))
print("The approximation ratio of our solution in 2.c to the optimal value is: " + str(ratio))

The optimal value for Question 1 using 2.a is 2.2499999998622675
A solution X is
[[ 1.  -0.5 -0.5]
 [-0.5  1.  -0.5]
 [-0.5 -0.5  1. ]]
The optimal value of the SDP program in 2.c is  368.2732524570028
The optimal value of the MAXCUT in 2.c is: 333.0
The approximation ratio of our solution in 2.c to the optimal value is: 0.90421989047081


In [4]:
#'G' is given as an adjaceny matrix
adj_mat = [[   0.  , 2., 1000.  ,  2. , 1000.  ,  2.  ,  2. ,1000.  ,  2.  ,  2.],
 [ 2.  ,  0. ,1000.  ,  2., 1000.  ,  2.  ,  2. ,1000.  ,  2.  ,  2.],
 [1000. , 1000.  ,  0. ,1000. ,   2. ,1000., 1000. ,   2. ,1000., 1000.],
 [   2.  ,  2. ,1000.  ,  0. ,1000.  ,  2. ,   2., 1000.  ,  2.  ,  2.],
 [1000., 1000.  ,  2. ,1000. ,   0., 1000. ,1000.  ,  2. ,1000. ,1000.],
 [   2.  ,  2., 1000.  ,  2., 1000. ,   0. ,   2., 1000. ,   2. ,   2.],
 [   2.  ,  2. ,1000.  ,  2. ,1000. ,   2. ,   0., 1000. ,   2. ,   2.],
 [1000.  ,1000.,    2. ,1000.,    2., 1000., 1000.,    0., 1000., 1000.],
 [   2. ,   2. ,1000. ,   2. ,1000. ,   2. ,   2., 1000.,    0.,    2.],
 [   2. ,   2. ,1000. ,   2. ,1000. ,   2. ,   2., 1000.,    2.,    0.]]


In [2]:
#2.d
def toEdgesV2(adj_mat):
    m = np.matrix(adj_mat)
    row = m.shape[0]
    col = m.shape[1]
    edges = []
    for i in range(row):
        for j in range(i, col):
            if m[i, j] != 0:
                edges.append(((i, j), m[i, j]))       
    return edges

def objectiveValueMatrixV2(n, edges):
    a = np.zeros((n, n), float)
    for i in range(len(edges)):
        row = edges[i][0][0]
        col = edges[i][0][1]
        weight = edges[i][1]
        a[row, col] = weight / 2
    return a

def constantsOfObjectiveValue(edges):
    v = 0
    for i in range(len(edges)):
        weight = edges[i][1]
        v += (-1) * weight / 2
    return v
    
# the implementation of SDP relaxation for MAXCUT in CVXPY
def sdp_maxcutV2(n, edges):
    X = cp.Variable((n,n), symmetric=True)
    constraints = [X >> 0]
    constraints += [cp.trace(singleNonZeroMatrix(i, n) @ X) == 1.0 for i in range(n)]
    C = objectiveValueMatrixV2(n, edges)
    conv = constantsOfObjectiveValue(edges)
    prob = cp.Problem(cp.Minimize(cp.trace(C @ X) + conv),
                  constraints)
    prob.solve()
    return prob, X

def calCutV2(edges, vertices):
    v = 0
    for i in range(len(edges)):
        edge = edges[i]
        weight = edge[1]
        v += (1 - vertices[edge[0][0]] * vertices[edge[0][1]]) * weight / 2
    return v

In [5]:
def maxCutV2():
    size = len(adj_mat)
    edges = toEdgesV2(adj_mat)    
    prob, X = sdp_maxcutV2(size, edges)
    print("The optimal value of the SDP program in 2.d is ", (-1) * prob.value)

    U = decompose(X)
    val = 0
    ratio = None
    ite = 10
    for i in range(ite):
        p = vectorNorm(size)
        vertices = rounding(U, p)
        val += calCutV2(edges, vertices)
    val = val / ite
    ratio = val / ((-1) * prob.value)
    return val, ratio

val, ratio = maxCutV2()
print("The optimal value of the MAXCUT in 2.d is: " + str(val))
print("The approximation ratio of our solution in 2.d to the optimal value is: " + str(ratio))

The optimal value of the SDP program in 2.d is  20999.997486451633
The optimal value of the MAXCUT in 2.d is: 21000.0
The approximation ratio of our solution in 2.d to the optimal value is: 1.0000001196927937


### Steps for implementing SDP for Maxcut

#### Formulate the Maxcut problem as SDP (recall lecture or refer to reference book [GM] in syllabus section)

####  Once you obtain 'X' (the solution positive-semi-definite matrix from SDP, you may use Cholesky decomposition to get the solution unit-vectors, stacked as column vectors in 'U')

```python
Xnew = X.value
eigs = np.linalg.eigh(Xnew)[0]
if np.min(eigs) < 0:
  Xnew = Xnew + (1.00001 * abs(min(eigs)) * np.identity(n_nodes))
elif np.min(eigs) == 0:
  Xnew = Xnew + 0.0000001 * np.identity(n_nodes)
U = np.linalg.cholesky(Xnew).T
```

#### Round the unit-vectors to appropriate partition as explained in [GM] or class.